In [2]:
import numpy as np
import pandas as pd
import csv
import openpyxl
import math
'''train = np.array([
    [0,0,1,2,3],
    [1,1,2,3,4],
    [2,2,3,4,5],
    [3,3,4,5,6]
]
)'''
'''
train = np.array([
    [0,1,1,1,3],
    [1,1,1,1,4],
    [2,1,1,1,5],
    [3,1,1,1,6]
]
)'''
train = pd.read_csv('train.csv').as_matrix()#read train.csv file
#print(type(train))
#xtrain=train[:,1:4]
#xtrain = xtrain/5
#true_label=train[:,4:5]
#print((xtrain),(true_label))


C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [3]:
def encode_into_prob(x,max_val):
    len=x.size
    y_list=[]
    for i in range(len):
        temp=[]
        for j in range(max_val):
            if(x[i]==j):
                temp.append(1)
            else:
                temp.append(0)
            #print(temp)
        y_list.append(temp)
    return y_list
        

In [4]:
xtrain=train[:,1:785]

xmean = np.mean(xtrain , axis=1)
means_expanded = np.outer(xmean, np.ones(784))
#print(means_expanded)
xtrain = xtrain - means_expanded

xstd = np.std(xtrain , axis=1)
std_expanded = np.outer(xstd, np.ones(784))
#print(std_expanded)
xtrain = xtrain*1.0/std_expanded 

true_label = train[:,785:]
y_true=np.array(encode_into_prob(true_label,10))
y_true=y_true.transpose()

#print(type(y_true))
#print(y_true.transpose())
print(xtrain)
print((y_true))

[[-0.55945675 -0.55945675  2.06254185 ...  2.0522595  -0.55945675
  -0.55945675]
 [-0.5380259  -0.5380259  -0.5380259  ... -0.5380259  -0.5380259
  -0.5380259 ]
 [-0.6486501  -0.6486501  -0.6486501  ... -0.6486501  -0.6486501
  -0.6486501 ]
 ...
 [-0.64973918 -0.64973918 -0.64973918 ... -0.64973918 -0.64973918
  -0.64973918]
 [-0.59745866 -0.59745866 -0.59745866 ... -0.59745866 -0.59745866
  -0.59745866]
 [-0.62629522 -0.62629522 -0.62629522 ... -0.62629522 -0.62629522
  -0.62629522]]
[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]


In [5]:
def feedforward(batch_train,hidden_layers):
     
    global weight_matrix_list
    global bias_list
    global preactivation_list 
    preactivation_list=[]
    global activation_list
    activation_list=[]
    global y_hat
    y_hat=[]
    a_new=[]
    a_last_layer=[]
    nos_data = len(batch_train)
    #print(nos_data)
    h=batch_train.transpose()
    preactivation_list.append(h)
    activation_list.append(h)
    #print(h)
    
#     print("y_hat in feed \n",y_hat )
    
    for j in range(hidden_layers):
        
#         print(".............................")
#         print("activation for layer ", j)
#         print(activation_list[j] , activation_list[j].shape)
#         print("weight",j+1)
#         print(weight_matrix_list[j], weight_matrix_list[j].shape)
#         print("bias ",j+1)
#         print(bias_list[j],bias_list[j].shape)
        
        a_new = bias_list[j]+np.matmul(weight_matrix_list[j].transpose(),activation_list[j])
        preactivation_list.append(a_new)
        
#         print("preactivation for layer",j+1)
#         print(preactivation_list[j+1], preactivation_list[j+1].shape)
        
        h = 1.0/(1.0+np.exp(-1*preactivation_list[j+1]))
        activation_list.append(h)

    
    j=hidden_layers
#     print(".............................")
#     print("activation for layer ", j)
#     print(activation_list[j] , activation_list[j].shape)
#     print("weight",j+1)
#     print(weight_matrix_list[j], weight_matrix_list[j].shape)
#     print("bias ",j+1)
#     print(bias_list[j],bias_list[j].shape)
    
    a_last_layer = bias_list[j]+np.matmul(weight_matrix_list[j].transpose(),activation_list[j])
    preactivation_list.append(a_last_layer)
    
#     print("last activation")
#     print(a_last_layer,a_last_layer.shape)

    y_hat = np.array(output(a_last_layer,batch_train))
    y_hat = y_hat.transpose()
    
#     print("y_hat in feedforward end")
#     print(y_hat)
    
    
def output(a_last_layer,batch_train):
    y_hat_initial=[]
    y_hat_local=[]
    nos_data=len(batch_train)
    for i in range(nos_data):
        y_hat_initial.append(softmax(a_last_layer[:,i:i+1].transpose()))

    length = len(y_hat_initial)
    for i in range(length):
        y_hat_local.append(y_hat_initial[i][0])
    
#     print("y_hat_local", y_hat_local)
#     print("y_hat_initial",y_hat_initial)
    return y_hat_local

def softmax(row_vector):
    sum_all=0
    row_vector = np.exp(row_vector)
    sum_all = np.sum(row_vector , axis=1)         #print("row_vector", row_vector)
    row_vector = (row_vector*1.0/sum_all)         #print("sum_all",sum_all)
    return row_vector

In [6]:
def backpropagation(y_true_batch):
    
#     print("weigt in backpropogation", weight_matrix_list)
#     print("...........y_true_batch..........\n",y_true_batch)
    
    global grad_a_k_list
    grad_a_k_list=[]
    
    global grad_w_k_list
    grad_w_k_list=[]
    
    global grad_h_k_list
    grad_h_k_list=[]
    
    global grad_b_k_list
    grad_b_k_list=[]
    
    global grad_b
    grad_b=[]
    
    global grad_g_dash_a_k_list
    grad_g_dash_a_k_list = []
    
    global grad_a_last
    grad_a_last=[]
    
    k = num_hidden+1
    
#     print("--------------------------in Backpropagation---------------------")
    
    grad_g_dash_a_k_list = g_dash(preactivation_list)
    
#     print("preactivation_list....................")
#     print("Nos. of matrices in preactivation_list is", len(preactivation_list))
#     print(preactivation_list)
#     print("g_dash..................")
#     print("Nos. of matrices in g_dash is ", len(grad_g_dash_a_k_list))
#     print(grad_g_dash_a_k_list)
    
    grad_a_last = -np.subtract(y_true_batch,y_hat)
    
#     print("y_true_batch \n", y_true_batch)
#     print("y_hat \n",y_hat)
    
    grad_a_k_list.insert(0,grad_a_last)
    
#     print("grad_a_last_layer", k)
#     print(grad_a_k_list[0],grad_a_k_list[0].shape)
  
    
    while(k>0):
        
#         print(".................grad_compute at layer",k)
        
        grad_w_k_list.insert(0,np.matmul(grad_a_k_list[0],activation_list[k-1].transpose()))
        
#         print("grad_a", k)
#         print(grad_a_k_list[0],grad_a_k_list[0].shape)
#         print('preactivation  a_',k-1)
#         print(preactivation_list[k-1],preactivation_list[k-1].shape)
#         print("grad_w_",k)
#         print(grad_w_k_list[0],grad_w_k_list[0].shape)
        
        grad_b_k_list.insert(0,grad_a_k_list[0])
        
#         print("grad_b",k)
#         print(grad_b_k_list[0],grad_b_k_list[0].shape)
        
        grad_h_k_list.insert(0,np.matmul(weight_matrix_list[k-1],grad_a_k_list[0]))
        
#         print("grad_w",k)
#         print(grad_w_k_list[0],grad_w_k_list[0].shape)
#         print("grad_a",k)
#         print(grad_a_k_list[0],grad_a_k_list[0].shape)
#         print("grad_h",k-1)
#         print(grad_h_k_list[0],grad_h_k_list[0].shape)
        
        grad_a_k_list.insert(0,np.multiply(grad_h_k_list[0],grad_g_dash_a_k_list[k-1]))
        
#         print("grad_h_k_list[0]",k-1)
#         print(grad_h_k_list[0],grad_h_k_list[0].shape)
#         print("grad_g_dash_a_k_list[k-1]",k-1)
#         print(grad_g_dash_a_k_list[k-1],grad_g_dash_a_k_list[k-1].shape)
#         print("grad_a",k-1)
#         print(grad_a_k_list[0],grad_a_k_list[0].shape)
        k=k-1
#         print("grad_a_ik_list",grad_a_k_list[0])
    
    sum_all_grad_b_of_data(grad_b_k_list)
    

def sum_all_grad_b_of_data(grad_b_k_list):
    length=len(grad_b_k_list)
    i=0
    while(i<length):
        row=[]
        row = np.sum(grad_b_k_list[i],1)
        j = len(row)
        #print(row.shape)
        row = row[:, np.newaxis]
        #print(row.shape , row)
        grad_b.append(row)
        i+=1
    #print("grad_b",grad_b)

def g_dash(preactivation_list):
    length=len(preactivation_list)
    a_modified = preactivation_list[0:length]
    g_dash_a_k_list=[]
    for i in range(length):
        a=(1.0/(1+np.exp(-a_modified[i])))*(1-(1.0/(1+np.exp(-a_modified[i]))))
        g_dash_a_k_list.append(a)
    return g_dash_a_k_list


In [7]:
def decode_yhat_to_classes(y_hat_local):
    col = len(y_hat_local[0])
    max_val=0
    for j in range(col):
        max_val = y_hat_local[0][j]
        index=0
        for i in range(9):
            if(y_hat_local[i+1][j] > max_val):
                y_hat_local[index][j]=0
                index = i+1
                max_val=y_hat[i+1][j]
            else :
                y_hat_local[i+1][j]=0
        y_hat_local[index][j]=1
    return y_hat_local

In [8]:
def classifcation(y_hat):
    y_predict=[]
    col = len(y_hat[0])
    for j in range(col):
        for i in range(10):
            if(y_hat[i][j]==1):
                y_predict.append(i+1)
                break
    return np.array(y_predict)

In [9]:
#ip_neurons = 3 #input layer
ip_neurons = 784
num_hidden = 2
encoding_bits=10
sizes = [50,100]
sizes.insert(0,ip_neurons)
sizes.append(encoding_bits)
print(sizes)
print(len(sizes))

weight_matrix_list = []

bias_list =[]
y_hat= []

for i in range(int(num_hidden)+1):
    weight_matrix_list.append(np.random.rand(sizes[i],sizes[i+1])/np.sqrt(sizes[i]))
    bias_list.append(0*np.random.rand(sizes[i+1],1))
print(weight_matrix_list)
#print(len(weight_matrix_list))

print(bias_list)
#print(hlist)



[784, 50, 100, 10]
4
[array([[0.02877622, 0.00612583, 0.01060217, ..., 0.02232832, 0.01609823,
        0.03240143],
       [0.01916133, 0.02359844, 0.01224246, ..., 0.03169097, 0.01498393,
        0.02119273],
       [0.0004331 , 0.03338946, 0.01581992, ..., 0.00632437, 0.0148531 ,
        0.03523695],
       ...,
       [0.01248194, 0.00885848, 0.03438131, ..., 0.0148197 , 0.00449166,
        0.02872777],
       [0.02863293, 0.02627378, 0.02268934, ..., 0.01922257, 0.0145979 ,
        0.00309374],
       [0.00080867, 0.00818828, 0.03267851, ..., 0.01728382, 0.02359067,
        0.00057805]]), array([[0.0704615 , 0.13309439, 0.10910513, ..., 0.10945406, 0.13304574,
        0.08807632],
       [0.07408403, 0.11719756, 0.1272977 , ..., 0.11134738, 0.09231599,
        0.00299497],
       [0.00287579, 0.13030375, 0.02938027, ..., 0.10366056, 0.00156677,
        0.10191214],
       ...,
       [0.09350648, 0.12471002, 0.07459373, ..., 0.12564391, 0.01299333,
        0.13148649],
       [0.03

In [14]:
def grad_descent(weight_matrix_list,bias_list,xtrain,num_hidden,sizes):
    
    len_w = len(weight_matrix_list)
    len_b = len(bias_list)
    data_length = len(xtrain)
    batch_size = 10
    eta=0.01
    no_of_batch = int(data_length/batch_size)
    epochs =30
    steps = 0
    loss=0
    error = 0
    nos_points = 0
    file = open('log_file.txt', 'w')
    for t in range(epochs):
        loss=0
        error=0
        for batch in range(no_of_batch):
            dw=[]
            db=[]
            for i in range(int(num_hidden)+1):
                dw.append(0*np.random.rand(sizes[i],sizes[i+1]))
                db.append(0*np.random.rand(sizes[i+1],1))
            
            for data_index_in_batch in range(batch_size):
                l=batch_size*batch + data_index_in_batch
                r=batch_size*batch + data_index_in_batch + 1
#                 print("before\n",data_index_in_batch)
                feedforward(xtrain[l:r,:],num_hidden)
#                 print("after\n",y_hat)
#                 print(l , r)
                backpropagation(y_true[:,l:r])

                for j in range(len_w):
                    dw[j] += grad_w_k_list[j].transpose()

                for j in range(len_b):
                    db[j] += grad_b[j]
                
                
                loss += cal_loss(y_hat,y_true[:,l:r])
                
                error += cal_error(y_hat,y_true[:,l:r])
                
                nos_points +=1
                
            for j in range(len_w):
                weight_matrix_list[j] = weight_matrix_list[j] - eta*dw[j]

            for j in range(len_b):
                bias_list[j] =  bias_list[j] - eta*db[j]
            
            if (steps%300==0):
                file.write("Epoch "+str(t+1)+", Step "+str(steps)+", loss : "+str(loss)+", error : "+str(error/nos_points)+", lr: " + str(eta)+ "\n")
#                 print("loss \n",loss)
            steps += 1
            
            
#             print("after batch ", batch)
#             print("y_hat", y_hat)
            
#             print("subtracted weight\n", weight_matrix_list)
            
            
            
        print("end  epochs :  ", t, loss)
#     file.close()

In [15]:
def cal_loss(y_hat,y_true):
    loss = 0
    for i in range(10):
        if(y_hat[i]!=0):
            loss += (-1)*y_true[i]*math.log(y_hat[i]) 
    
    return loss
        
def cal_error(y_hat,y_true):
    true = classifcation(y_true)
    predicted = classifcation(decode_yhat_to_classes(y_hat))
    if (true != predicted):
        return 1
    return 0

In [16]:
def classifcation(y_hat):
    y_predict=[]
    col = len(y_hat[0])
    for j in range(col):
        for i in range(10):
            if(y_hat[i][j]==1):
                y_predict.append(i)
                break
    return np.array(y_predict)

In [17]:
grad_descent(weight_matrix_list,bias_list,xtrain,num_hidden,sizes)
#print(y_hat)0
#print(len(xtrain))

end  epochs :   0 [85738.54342296]
end  epochs :   1 [30845.12313987]
end  epochs :   2 [21132.8001389]
end  epochs :   3 [15954.9657875]
end  epochs :   4 [12287.05346344]
end  epochs :   5 [9576.26599874]
end  epochs :   6 [7443.87721191]
end  epochs :   7 [5891.19232748]
end  epochs :   8 [4702.44427428]
end  epochs :   9 [3754.8230568]
end  epochs :   10 [3020.75943143]
end  epochs :   11 [2412.91866696]
end  epochs :   12 [1888.71289096]
end  epochs :   13 [1496.76798849]
end  epochs :   14 [1227.93940742]
end  epochs :   15 [1050.52608523]
end  epochs :   16 [920.22328146]
end  epochs :   17 [811.95643289]
end  epochs :   18 [717.01776418]
end  epochs :   19 [652.68604795]
end  epochs :   20 [587.88960546]
end  epochs :   21 [536.45091292]
end  epochs :   22 [490.61476965]
end  epochs :   23 [454.75237784]
end  epochs :   24 [421.2531417]
end  epochs :   25 [393.10175443]
end  epochs :   26 [366.42889894]
end  epochs :   27 [340.99526414]
end  epochs :   28 [312.48752092]
end  ep

In [18]:
feedforward(xtrain,num_hidden)
print(y_hat)
y_hat = decode_yhat_to_classes(y_hat)
print(y_hat)

[[1.75932665e-14 3.90802496e-14 9.74416644e-10 ... 1.67696490e-12
  6.92030715e-20 5.91251113e-20]
 [1.41370952e-06 1.17046026e-13 9.99999788e-01 ... 9.95695069e-10
  5.98762419e-14 2.11174747e-16]
 [1.79138974e-16 5.63427164e-06 4.49225672e-10 ... 7.10184244e-05
  2.82842226e-09 1.50243261e-11]
 ...
 [2.97640676e-11 1.29621418e-10 3.82646144e-10 ... 1.77283659e-08
  2.56000661e-19 5.79414405e-11]
 [8.47558760e-15 3.95143768e-11 6.66210328e-19 ... 6.16514117e-10
  3.19123190e-17 8.68294014e-07]
 [1.40447280e-10 5.94802479e-04 2.10099484e-07 ... 3.23330801e-08
  9.99999965e-01 1.31642835e-11]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [19]:
def classifcation(y_hat):
    y_predict=[]
    col = len(y_hat[0])
    for j in range(col):
        for i in range(10):
            if(y_hat[i][j]==1):
                y_predict.append(i)
                break
    return np.array(y_predict)
y_predict = classifcation(y_hat)
print((y_predict))
y_predict = y_predict[:, np.newaxis]
print((y_predict))
print(true_label)

[4 6 1 ... 6 9 6]
[[4]
 [6]
 [1]
 ...
 [6]
 [9]
 [6]]
[[4.]
 [6.]
 [1.]
 ...
 [6.]
 [9.]
 [6.]]


In [20]:
cnt=0
for i in range(55000):
    if(y_predict[i]==true_label[i]):
        cnt+=1
acc = cnt/55000
print(acc)

0.9993454545454545


In [21]:
valid = pd.read_csv('valid.csv').as_matrix()
xtrain=valid[:,1:785]

xmean = np.mean(xtrain , axis=1)
means_expanded = np.outer(xmean, np.ones(784))
#print(means_expanded)
xtrain = xtrain - means_expanded

xstd = np.std(xtrain , axis=1)
std_expanded = np.outer(xstd, np.ones(784))
#print(std_expanded)
xtrain = xtrain*1.0/std_expanded 

true_label = valid[:,785:]


C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [22]:
feedforward(xtrain,num_hidden)
print(y_hat)
y_hat = decode_yhat_to_classes(y_hat)
print(y_hat)

[[3.93834554e-11 1.02468803e-17 1.78979642e-16 ... 9.86281578e-01
  1.58641175e-11 1.85552545e-06]
 [9.89884136e-01 3.48945317e-12 5.74066579e-09 ... 2.35231489e-07
  6.59855136e-12 1.38623127e-09]
 [1.35031331e-05 1.00571421e-11 5.87773320e-14 ... 5.44747967e-06
  1.27070025e-09 9.99973733e-01]
 ...
 [6.17277149e-08 2.65297674e-13 2.45942419e-11 ... 2.84401162e-12
  5.03711585e-09 4.47545802e-09]
 [5.80139558e-08 2.21156031e-18 4.63412291e-10 ... 2.15909371e-11
  2.57953402e-02 1.13254908e-07]
 [1.29953666e-03 9.99999621e-01 1.39413547e-10 ... 7.77268318e-12
  3.53749779e-15 6.79707154e-08]]
[[0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [23]:
def classifcation(y_hat):
    y_predict=[]
    col = len(y_hat[0])
    for j in range(col):
        for i in range(10):
            if(y_hat[i][j]==1):
                y_predict.append(i)
                break
    return np.array(y_predict)
y_predict = classifcation(y_hat)
print((y_predict))
y_predict = y_predict[:, np.newaxis]
print((y_predict))
print(true_label)

[1 9 4 ... 0 6 2]
[[1]
 [9]
 [4]
 ...
 [0]
 [6]
 [2]]
[[1.]
 [9.]
 [4.]
 ...
 [0.]
 [6.]
 [2.]]


In [24]:
cnt=0
k = len(true_label)
for i in range(k):
    if(y_predict[i]==true_label[i]):
        cnt+=1
acc = cnt/k
print(acc)

0.8428


In [25]:

xtest = pd.read_csv('test.csv').as_matrix()
test=xtest[:,1:]

xmean = np.mean(test , axis=1)
means_expanded = np.outer(xmean, np.ones(784))
#print(means_expanded)
test = test - means_expanded

xstd = np.std(test , axis=1)
std_expanded = np.outer(xstd, np.ones(784))
#print(std_expanded)
test = test*1.0/std_expanded 

feedforward(test,num_hidden)
#print(y_hat)
# y_hat = decode_yhat_to_classes(y_hat)
#print(y_hat)

C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [26]:
print(y_hat)
y_hat = decode_yhat_to_classes(y_hat)
print(y_hat)

[[4.53762045e-10 2.77618754e-11 1.05436853e-12 ... 2.44743015e-10
  2.60388112e-06 1.82295881e-12]
 [7.73721477e-11 3.00693989e-11 5.38004485e-07 ... 6.65360084e-07
  1.58629553e-07 9.99999999e-01]
 [9.86044509e-01 7.58832220e-05 2.68404477e-04 ... 8.63218592e-05
  3.88119656e-02 4.18564324e-17]
 ...
 [2.14408314e-06 1.26038164e-06 5.34429980e-11 ... 4.80418072e-14
  4.26157543e-05 1.62400035e-11]
 [7.41838853e-03 2.72672255e-01 7.86506236e-06 ... 1.29930392e-15
  9.61050160e-01 6.56175854e-22]
 [5.08682570e-03 9.60863305e-08 9.98863424e-01 ... 9.90874369e-01
  2.27727935e-08 4.49782196e-11]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 1. 0. 0.]]


In [27]:
def classifcation(y_hat):
    y_predict=[]
    col = len(y_hat[0])
    for j in range(col):
        for i in range(10):
            if(y_hat[i][j]==1):
                y_predict.append(i)
                break
    return np.array(y_predict)

In [28]:
y_predict = classifcation(y_hat)
print((y_predict))
y_predict = y_predict[:, np.newaxis]
print((y_predict))

[2 6 9 ... 9 8 1]
[[2]
 [6]
 [9]
 ...
 [9]
 [8]
 [1]]


In [29]:
def add_id_to_y_predict(y_predict):
    ide=[]
    row=len(y_predict)
    for i in range(row):
        ide.append(i)
    ide=np.array(ide)
    ide=ide[:,np.newaxis]
    y= np.hstack((ide,y_predict))
    #print(y_predict, ide)
    return y           

In [30]:
print(y_predict)
#y_predict= np.random.randint(10, size=(10000, 1))
y_predict= add_id_to_y_predict(y_predict)
print(y_predict)
x,y = y_predict.shape
#print(x,y)

[[2]
 [6]
 [9]
 ...
 [9]
 [8]
 [1]]
[[   0    2]
 [   1    6]
 [   2    9]
 ...
 [9997    9]
 [9998    8]
 [9999    1]]


In [31]:
with open('predict_test_today.csv', 'w', newline='') as f:
    thewriter = csv.writer(f)
    row,col=y_predict.shape 
    thewriter.writerow(['id','label'])
    for i in range(row):
        thewriter.writerow(y_predict[i])
#out.close()
#np.savetxt("foo.csv", y_predict, delimiter=",")